In [ ]:
# imports
!pip install pytest ipytest

In [ ]:
# classification model
from google import genai
from google.genai import types
import base64

def classification_model(user_input):
  client = genai.Client(
      vertexai=True,
      project="qwiklabs-gcp-01-3005c83a3aae",
      location="us-central1",
  )

  si_text1 = """You are a classification model. Classify the user's question into one of the following categories: Employment, General Information, Emergency Services, or Tax Related. Only return the related category."""

  model = "gemini-2.0-flash-lite-001"
  # model = "gemini-2.5-pro-preview-03-25"
  contents = [
    types.Content(
      role="user",
      parts=[
        types.Part.from_text(text=user_input)
      ]
    ),
  ]
  generate_content_config = types.GenerateContentConfig(
    temperature = 1,
    top_p = 0.95,
    seed = 0,
    max_output_tokens = 1346,
    response_modalities = ["TEXT"],
    safety_settings = [types.SafetySetting(
      category="HARM_CATEGORY_HATE_SPEECH",
      threshold="BLOCK_LOW_AND_ABOVE"
    ),types.SafetySetting(
      category="HARM_CATEGORY_DANGEROUS_CONTENT",
      threshold="BLOCK_LOW_AND_ABOVE"
    ),types.SafetySetting(
      category="HARM_CATEGORY_SEXUALLY_EXPLICIT",
      threshold="BLOCK_LOW_AND_ABOVE"
    ),types.SafetySetting(
      category="HARM_CATEGORY_HARASSMENT",
      threshold="BLOCK_LOW_AND_ABOVE"
    )],
    system_instruction=[types.Part.from_text(text=si_text1)],
  )

  response = client.models.generate_content(
          model=model,
          contents=contents,
          config = generate_content_config,
        )
  # print("response:",response.text)
  return response.text.strip()

#user question
user_input = "can I get some help with taxes"


print(f"Question: {user_input}\n--------------------------------\n\n")
print(classification_model(user_input))

Question: can I get some help with taxes
--------------------------------


Tax Related


In [ ]:
# classification unit tests
import ipytest
ipytest.autoconfig()

def test_employment_classification():
    assert classification_model("I need help with my resume") == "Employment"

def test_general_info_classification():
    assert classification_model("what is the phone number for appointments") == "General Information"

def test_emergency_services_classification():
    assert classification_model("can I urgent help") == "Emergency Services"

def test_tax_related_classification():
    assert classification_model("can I get some help with taxes") == "Tax Related"

ipytest.run("-vv")

======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-8.3.4, pluggy-1.5.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content
plugins: typeguard-4.4.1, anyio-3.7.1
collecting ... collected 5 items

t_b1b35109246a46fcb522dde59b084480.py::test_addition PASSED                                  [ 20%]
t_b1b35109246a46fcb522dde59b084480.py::test_employment_classification PASSED                 [ 40%]
t_b1b35109246a46fcb522dde59b084480.py::test_general_info_classification PASSED               [ 60%]
t_b1b35109246a46fcb522dde59b084480.py::test_emergency_services_classification PASSED         [ 80%]
t_b1b35109246a46fcb522dde59b084480.py::test_tax_related_classification PASSED                [100%]

======================================== 5 passed in 1.26s =========================================


<ExitCode.OK: 0>

In [27]:
# social media post generator
from google import genai
from google.genai import types
import base64

def social_media_generator_model(user_input):
  client = genai.Client(
      vertexai=True,
      project="qwiklabs-gcp-01-3005c83a3aae",
      location="us-central1",
  )

  si_text1 = """You are a seasoned social media copy writer, write professional social media posts for government announcements on the topic given by the user. Ensure you include #s at the end of the posts to gain traction"""

  model = "gemini-2.0-flash-lite-001"
  # model = "gemini-2.5-pro-preview-03-25"
  contents = [
    types.Content(
      role="user",
      parts=[
        types.Part.from_text(text=user_input)
      ]
    ),
  ]
  generate_content_config = types.GenerateContentConfig(
    temperature = 1,
    top_p = 0.95,
    seed = 0,
    max_output_tokens = 1346,
    response_modalities = ["TEXT"],
    safety_settings = [types.SafetySetting(
      category="HARM_CATEGORY_HATE_SPEECH",
      threshold="BLOCK_LOW_AND_ABOVE"
    ),types.SafetySetting(
      category="HARM_CATEGORY_DANGEROUS_CONTENT",
      threshold="BLOCK_LOW_AND_ABOVE"
    ),types.SafetySetting(
      category="HARM_CATEGORY_SEXUALLY_EXPLICIT",
      threshold="BLOCK_LOW_AND_ABOVE"
    ),types.SafetySetting(
      category="HARM_CATEGORY_HARASSMENT",
      threshold="BLOCK_LOW_AND_ABOVE"
    )],
    system_instruction=[types.Part.from_text(text=si_text1)],
  )

  response = client.models.generate_content(
          model=model,
          contents=contents,
          config = generate_content_config,
        )
  # print("response:",response.text)
  return response.text.strip()


user_input = "Write a email about emergency rain levels forecasted tomorrow"


print(f"topic: {user_input}\n--------------------------------\n\n")
print(social_media_generator_model(user_input))

topic: Write a email about emergency rain levels forecasted tomorrow
--------------------------------


Okay, here are some social media posts you can adapt for government announcements about forecasted emergency rain levels tomorrow, along with some email content you can use to distribute this information:

**Social Media Posts (Adapt these to your specific platform and audience):**

**Post 1 (Focus: Alert and Urgency)**

"URGENT WEATHER ALERT! Heavy rainfall is forecasted for tomorrow, [Date], with potential for [mention specific hazards: flooding, flash floods, etc.] in [affected areas].  Stay informed and take precautions!  Monitor official sources for updates: [Link to government website/social media].  #EmergencyAlert #RainfallWarning #[YourCity/Region] #WeatherAwareness #StaySafe"

**Post 2 (Focus: Specific Actions to Take)**

"Prepare for heavy rain tomorrow, [Date]!  Here are some crucial steps to take:  Secure loose outdoor objects, clear drains and gutters, and avoid unneces

In [26]:
# post generator unit tests
# google evaluation api evaluation
from vertexai.evaluation import (
    MetricPromptTemplateExamples,
    EvalTask,
    PairwiseMetric,
    PairwiseMetricPromptTemplate,
    PointwiseMetric,
    PointwiseMetricPromptTemplate,
)
import datetime
import pandas as pd
from vertexai.generative_models import GenerativeModel

def social_media_generator_model_evaluation():

  topics = [
      "Write a email about emergency rain levels forecasted tomorrow",
      "Write a tweet about heavy traffic due to road closures next week",
      "Write an instagram caption about the new mayor, Doug"
  ]

  results = []

  for topic in topics:
    result = social_media_generator_model(topic)
    results.append(result)

  eval_dataset = pd.DataFrame({
    "prompt": topics,
  })

  eval_task = EvalTask(
    dataset=eval_dataset,
    metrics=[MetricPromptTemplateExamples.Pointwise.GROUNDEDNESS,
             MetricPromptTemplateExamples.Pointwise.COHERENCE
             ],
    experiment="apartment-listing-generation",
  )

  model = GenerativeModel(
    "gemini-2.0-flash-001",
    generation_config={
        "temperature": 0,
        "top_p": 0.4,
    },
  )

  run_ts = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  eval_result = eval_task.evaluate(
    model=model,
    experiment_run_name=f"apt-gen-{run_ts}"
    )
  eval_result.metrics_table
  # print(eval_result.metrics_table)
  print("eval obj",eval_result)

social_media_generator_model_evaluation()



INFO:google.cloud.aiplatform.metadata.experiment_resources:Associating projects/953893243677/locations/us-central1/metadataStores/default/contexts/apartment-listing-generation-apt-gen-20250428-210628 to Experiment: apartment-listing-generation


INFO:vertexai.evaluation.eval_task:Logging Eval Experiment metadata: {'model_name': 'publishers/google/models/gemini-2.0-flash-001', 'temperature': 0, 'top_p': 0.4}
INFO:vertexai.evaluation._evaluation:Generating a total of 3 responses from Gemini model gemini-2.0-flash-001.
100%|██████████| 3/3 [00:04<00:00,  1.61s/it]
INFO:vertexai.evaluation._evaluation:All 3 responses are successfully generated from Gemini model gemini-2.0-flash-001.
INFO:vertexai.evaluation._evaluation:Multithreaded Batch Inference took: 4.834879709000234 seconds.
INFO:vertexai.evaluation._evaluation:Computing metrics with a total of 6 Vertex Gen AI Evaluation Service API requests.
100%|██████████| 6/6 [00:06<00:00,  1.02s/it]
INFO:vertexai.evaluation._evaluation:All 6 metric requests are successfully computed.
INFO:vertexai.evaluation._evaluation:Evaluation Took:6.151133413997741 seconds


eval obj EvalResult(summary_metrics={'row_count': 3, 'groundedness/mean': 0.3333333333333333, 'groundedness/std': 0.5773502691896258, 'coherence/mean': 5.0, 'coherence/std': 0.0}, metrics_table=                                              prompt  \
0  Write a email about emergency rain levels fore...   
1  Write a tweet about heavy traffic due to road ...   
2  Write an instagram caption about the new mayor...   

                                            response  \
0  Subject: URGENT: Emergency Rain Levels Forecas...   
1  Heads up, commuters! ⚠️ Expect heavy traffic n...   
2  Okay, here are a few Instagram caption options...   

                            groundedness/explanation  groundedness/score  \
0  The response includes information that was not...                 0.0   
1  The response contains information not provided...                 0.0   
2  The response is fully grounded, as all aspects...                 1.0   

                               coherence/explanatio

In [29]:
# post generator unit tests

def test_hashtag():
  topics = [
        "Write a email about emergency rain levels forecasted tomorrow",
        "Write a tweet about heavy traffic due to road closures next week",
        "Write an instagram caption about the new mayor, Doug"
    ]

  results = []

  for topic in topics:
    result = social_media_generator_model(topic)
    results.append(result)

  assert all('#' in r for r in results), \
         f"Assertion Failed: Not all results contain a hashtag ('#'). Failed results: {[r for r in results if '#' not in r]}"

ipytest.run("-vv")

======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-8.3.4, pluggy-1.5.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content
plugins: typeguard-4.4.1, anyio-3.7.1
collecting ... collected 6 items

t_b1b35109246a46fcb522dde59b084480.py::test_addition PASSED                                  [ 16%]
t_b1b35109246a46fcb522dde59b084480.py::test_employment_classification PASSED                 [ 33%]
t_b1b35109246a46fcb522dde59b084480.py::test_general_info_classification PASSED               [ 50%]
t_b1b35109246a46fcb522dde59b084480.py::test_emergency_services_classification PASSED         [ 66%]
t_b1b35109246a46fcb522dde59b084480.py::test_tax_related_classification PASSED                [ 83%]
t_b1b35109246a46fcb522dde59b084480.py::test_hashtag PASSED                                   [100%]

======================================== 6 passed in 12.38s ========================================


<ExitCode.OK: 0>